# Importaciones y Configuración de Modelos

In [1]:
import asyncio
import multiprocessing
from scapy.all import sniff, get_if_list
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import nest_asyncio
import matplotlib.pyplot as plt
import os

# Configurar los modelos
isolation_forest = IsolationForest()
one_class_svm = OneClassSVM()

# Funciones de Captura y Preprocesamiento de Paquetes

In [4]:
def capture_packets_live(queue, packet_count=100, iface='enp7s0'):
    print(get_if_list())  # Verifica las interfaces disponibles
    packets = sniff(filter="ip", count=packet_count, iface=iface)
    print(f"Capturados {len(packets)} paquetes en la interfaz {iface}")
    queue.put(packets)

def preprocess_live_packets(packets):
    packet_data = []
    for packet in packets:
        if 'IP' in packet:
            try:
                if packet.haslayer('TCP'):
                    protocol = 'TCP'
                    src_port = packet['TCP'].sport
                    dst_port = packet['TCP'].dport
                elif packet.haslayer('UDP'):
                    protocol = 'UDP'
                    src_port = packet['UDP'].sport
                    dst_port = packet['UDP'].dport
                else:
                    continue

                src_addr = int(''.join([f'{int(octet):08b}' for octet in packet['IP'].src.split('.')]), 2)
                dst_addr = int(''.join([f'{int(octet):08b}' for octet in packet['IP'].dst.split('.')]), 2)
                timestamp = packet.time

                packet_data.append({
                    'protocol': protocol,
                    'src_addr': src_addr,
                    'src_port': src_port,
                    'dst_addr': dst_addr,
                    'dst_port': dst_port,
                    'timestamp': timestamp
                })
            except Exception as e:
                print(f"Paquete ignorado debido a error: {e}")
        else:
            print("Paquete ignorado debido a falta de capa IP")

    df = pd.DataFrame(packet_data)
    print(f"Paquetes procesados: {len(df)}")
    if not df.empty:
        df['packet_size'] = df['src_port'] + df['dst_port']
        df['time_interval'] = df['timestamp'].diff().fillna(0)

        numeric_columns = ['src_addr', 'dst_addr', 'src_port', 'dst_port', 'packet_size', 'time_interval']
        df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(df[numeric_columns])
        return df_scaled
    else:
        return None

def detect_anomalies_live(df_scaled):
    if df_scaled is not None:
        anomalies_if = isolation_forest.fit_predict(df_scaled)
        anomalies_svm = one_class_svm.fit_predict(df_scaled)
        return anomalies_if, anomalies_svm
    else:
        return [], []


# Función para Graficar Anomalías y Captura Asíncrona de Paquetes

In [5]:
def plot_anomalies(anomalies_if, anomalies_svm):
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))

    ax[0].hist(anomalies_if, bins=2, edgecolor='black')
    ax[0].set_title('Isolation Forest Anomalies')
    ax[0].set_xticks([-1, 1])
    ax[0].set_xticklabels(['Anomalía', 'Normal'])

    ax[1].hist(anomalies_svm, bins=2, edgecolor='black')
    ax[1].set_title('One-Class SVM Anomalies')
    ax[1].set_xticks([-1, 1])
    ax[1].set_xticklabels(['Anomalía', 'Normal'])

    fig.tight_layout()

    # Asegurarnos de que la ruta /tmp exista
    if not os.path.exists('/tmp'):
        os.makedirs('/tmp')

    plt.savefig('/tmp/anomalies.png')
    plt.close()

async def async_capture_packets(queue, packet_count=100, iface='enp7s0'):
    loop = asyncio.get_event_loop()
    packets = await loop.run_in_executor(None, lambda: sniff(filter="ip", count=packet_count, iface=iface))
    print(f"Capturados {len(packets)} paquetes en la interfaz {iface}")
    await queue.put(packets)

async def async_main_live_detection(packet_count=100, iface='enp7s0'):
    queue = asyncio.Queue()
    await async_capture_packets(queue, packet_count, iface)

    packets = await queue.get()
    print(f"Primeros 5 paquetes capturados: {packets[:5]}")
    df_scaled = preprocess_live_packets(packets)
    anomalies_if, anomalies_svm = detect_anomalies_live(df_scaled)
    plot_anomalies(anomalies_if, anomalies_svm)
    return anomalies_if, anomalies_svm

def main_live_detection(packet_count=100, iface='enp7s0'):
    loop = asyncio.get_event_loop()
    if loop.is_running():
        nest_asyncio.apply(loop)
        anomalies_if, anomalies_svm = loop.run_until_complete(async_main_live_detection(packet_count, iface))
    else:
        anomalies_if, anomalies_svm = loop.run_until_complete(async_main_live_detection(packet_count, iface))
    return anomalies_if, anomalies_svm

if __name__ == "__main__":
    try:
        anomalies_if, anomalies_svm = main_live_detection(100, iface='enp7s0')  # Ajusta 'enp7s0' a tu interfaz de red activa
        print("Anomalías detectadas por Isolation Forest:", anomalies_if)
        print("Anomalías detectadas por One-Class SVM:", anomalies_svm)
    except ValueError as e:
        print(e)


Capturados 100 paquetes en la interfaz enp7s0
Primeros 5 paquetes capturados: <mod Sniffed: TCP:5 UDP:0 ICMP:0 Other:0>
Paquetes procesados: 87
Anomalías detectadas por Isolation Forest: [ 1  1  1  1  1  1 -1 -1 -1 -1  1  1 -1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1  1  1  1  1  1  1  1
 -1 -1 -1 -1 -1 -1  1 -1  1  1  1 -1  1  1  1]
Anomalías detectadas por One-Class SVM: [ 1  1  1  1  1  1 -1 -1 -1 -1  1  1 -1  1 -1  1  1  1 -1  1  1  1 -1  1
  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1  1 -1  1  1 -1  1  1
  1 -1  1  1  1  1  1 -1  1  1  1 -1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


# Integración con Flask para Visualización en Tiempo Real

In [6]:
from flask import Flask, render_template_string, jsonify, send_file
import threading
import time

# Configurar Flask
app = Flask(__name__)

# Variables globales para almacenar los resultados
anomalies_if = []
anomalies_svm = []
df_global = pd.DataFrame()

def run_detection():
    global anomalies_if, anomalies_svm, df_global
    while True:
        try:
            anomalies_if, anomalies_svm = main_live_detection(100, iface='enp7s0')
            df_global = pd.DataFrame({'IsolationForest': anomalies_if, 'OneClassSVM': anomalies_svm})
            time.sleep(10)  # Esperar 10 segundos antes de la siguiente captura
        except Exception as e:
            print(f"Error en la detección: {e}")

@app.route('/')
def index():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Detección de Anomalías en Tiempo Real</title>
        <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            h1 {
                text-align: center;
            }
            .anomalies {
                margin: 20px;
            }
            .anomaly-list {
                list-style-type: none;
                padding: 0;
            }
            .anomaly-item {
                padding: 10px;
                border: 1px solid #ddd;
                margin-bottom: 5px;
            }
        </style>
    </head>
    <body>
        <h1>Detección de Anomalías en Tiempo Real</h1>
        <div class="anomalies">
            <h2>Isolation Forest</h2>
            <ul id="if-anomalies" class="anomaly-list"></ul>
            <h2>One-Class SVM</h2>
            <ul id="svm-anomalies" class="anomaly-list"></ul>
        </div>
        <div>
            <h2>Gráficos de Anomalías</h2>
            <img src="/plot.png" alt="Anomalies">
        </div>
        <script>
            function fetchAnomalies() {
                $.getJSON('/anomalies', function(data) {
                    $('#if-anomalies').empty();
                    $('#svm-anomalies').empty();

                    data.IsolationForest.forEach(function(anomaly, index) {
                        $('#if-anomalies').append('<li class="anomaly-item">' + anomaly + '</li>');
                    });

                    data.OneClassSVM.forEach(function(anomaly, index) {
                        $('#svm-anomalies').append('<li class="anomaly-item">' + anomaly + '</li>');
                    });
                });
            }

            setInterval(fetchAnomalies, 5000);  // Actualizar cada 5 segundos
            fetchAnomalies();
        </script>
    </body>
    </html>
    ''')

@app.route('/anomalies')
def get_anomalies():
    global anomalies_if, anomalies_svm
    return jsonify({
        'IsolationForest': np.array(anomalies_if).tolist(),
        'OneClassSVM': np.array(anomalies_svm).tolist()
    })

@app.route('/plot.png')
def plot_png():
    return send_file('/tmp/anomalies.png', mimetype='image/png')

# Iniciar el hilo de detección en tiempo real
detection_thread = threading.Thread(target=run_detection)
detection_thread.start()

# Iniciar la aplicación Flask en un puerto diferente
from IPython.display import display, HTML

def run_flask():
    app.run(debug=False, host='0.0.0.0', port=8000)  # Desactivar el recargador automático

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Mostrar el enlace en Jupyter Notebook
display(HTML('<a href="http://localhost:8000" target="_blank">Haz clic aquí para abrir la interfaz web</a>'))


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.1.11:8000
Press CTRL+C to quit
192.168.1.11 - - [30/Jun/2024 00:28:24] "GET / HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:24] "GET /plot.png HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:24] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:30] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:35] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:40] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:45] "GET /anomalies HTTP/1.1" 200 -


Capturados 100 paquetes en la interfaz enp7s0
Primeros 5 paquetes capturados: <mod Sniffed: TCP:5 UDP:0 ICMP:0 Other:0>
Paquetes procesados: 89


192.168.1.11 - - [30/Jun/2024 00:28:50] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:28:55] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:00] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:05] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:10] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:15] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:20] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:25] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:30] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:35] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:40] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:45] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:50] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:29:55] "GET /anomalies HTTP/1.1

Capturados 100 paquetes en la interfaz enp7s0
Primeros 5 paquetes capturados: <mod Sniffed: TCP:4 UDP:1 ICMP:0 Other:0>
Paquetes procesados: 81


192.168.1.11 - - [30/Jun/2024 00:30:25] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:30:30] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:30:35] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:30:40] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:30:45] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:30:50] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:30:55] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:00] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:05] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:10] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:15] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:20] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:25] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:31:30] "GET /anomalies HTTP/1.1

Capturados 100 paquetes en la interfaz enp7s0
Primeros 5 paquetes capturados: <mod Sniffed: TCP:4 UDP:1 ICMP:0 Other:0>
Paquetes procesados: 71


192.168.1.11 - - [30/Jun/2024 00:31:55] "GET /anomalies HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 00:31:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 00:31:59] "GET /plot.png HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 00:31:59] "GET /anomalies HTTP/1.1" 200 -
192.168.1.11 - - [30/Jun/2024 00:32:00] "GET /anomalies HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 00:32:04] "GET /anomalies HTTP/1.1" 200 -


Capturados 100 paquetes en la interfaz enp7s0
Primeros 5 paquetes capturados: <mod Sniffed: TCP:4 UDP:1 ICMP:0 Other:0>
Paquetes procesados: 89
Capturados 100 paquetes en la interfaz enp7s0
Primeros 5 paquetes capturados: <mod Sniffed: TCP:2 UDP:3 ICMP:0 Other:0>
Paquetes procesados: 80
